# pyGVEC output for CAS3D

In [ ]:
from pathlib import Path
import datetime

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

import gvec
import gvec.fourier
import gvec.surface

In [ ]:
# run GVEC if not already done
gvec.run("parameter.ini")

In [ ]:
parameterfile = Path("parameter.ini")
params = gvec.util.read_parameter_file(parameterfile)
name = params["ProjectName"]
print(f"Project name: {name}")

statefile = sorted(Path(".").glob(f"{name}_State_*.dat"))[-1]
print(f"Found statefile: {statefile}")

In [ ]:
n_rho = 20
# radial_spacing = "rho^2"  # fixed!, linear spacing in rho^2
angles = "boozer"  # or "gvec"
MN_booz = 10, 10  # default: MN_out = MN_booz
MN_out = 10, 10
sampling = 8
stellsym = False

`gvec2cas3d ...`
`gvec_to_cas3d --rho=20 --angle=boozer --MN-booz="10,10" --MN-out="10,10"`
`gvec_to_cas3d --angle=boozer --MN-booz="10,10" --MN-out="10,10" --sampling=4`

In [ ]:
with gvec.State(parameterfile, statefile) as state:
    # Boozer transform
    rho = np.sqrt(np.linspace(0, 1.0, n_rho))
    rho[0] = 1e-4
    ev = gvec.EvaluationsBoozer(
        rho,
        sampling * MN_out[0] + 1,
        sampling * MN_out[1] + 1,
        M=MN_booz[0],
        N=MN_booz[1],
        state=state,
    )

    # Surface reparametrization
    state.compute(ev, "N_FP", "pos")
    surf = gvec.surface.init_surface(ev.pos, ev.N_FP, ift="fft")
    q_surf = [
        "xhat",
        "yhat",
        "zhat",
        "g_tt_B",
        "g_tz_B",
        "g_zz_B",
        "II_tt_B",
        "II_tz_B",
        "II_zz_B",
        "k_tt_B",
        "normal",
    ]
    gvec.surface.compute(surf, *q_surf)
    surf = surf[q_surf]

    # Quantities of interest (computed from equilibrium)
    q_vol = [
        "N_FP",
        "mod_B",
        "B_contra_t_B",
        "B_contra_z_B",
        "B_theta_avg",
        "B_zeta_avg",
        "iota",
        "p",
        "Phi",
        "Jac_B",
        "X1",
        "Jac_l",
        "Jac_h",
    ]
    state.compute(ev, *q_vol)
    ev = ev[q_vol]

    ds = xr.merge([ev, surf])

In [ ]:
ft = gvec.comp.ev2ft(ds)
ft = ft.where(
    (ft.m <= MN_out[0]) & (-MN_out[1] <= ft.n) & (ft.n <= MN_out[1]), drop=True
)
# ft = gvec.fourier.ft_autoremove(ft)
print(f"dataset uses {ft.nbytes / 1024**2:.2f} MiB")

In [ ]:
ft.attrs["gvec_version"] = gvec.__version__
ft.attrs["creator"] = "gvec_to_cas3d"
ft.attrs["arguments"] = repr(
    dict(n_rho=n_rho, MN_booz=MN_booz, MN_out=MN_out, sampling=sampling, angles=angles)
)
ft.attrs["statefile"] = statefile.name
ft.attrs["state_name"] = name
ft.attrs["conversion_time"] = (
    datetime.datetime.now().astimezone().isoformat(timespec="seconds")
)
ft.to_netcdf(f"{name}_BoozFT.nc")

In [ ]:
print(sampling)
np.abs(ft).max()

In [ ]:
import tqdm
import time

with tqdm.tqdm(
    total=10,
    bar_format="{n_fmt}/{total_fmt} |{bar}| {desc}",
    desc="Progress",
    ascii=True,
) as progress:
    progress.update(0)
    for _ in range(10):
        time.sleep(0.1)
    progress.update(1)
    progress.set_postfix(dict(stage="Testing"))
    for _ in range(10):
        time.sleep(0.1)
    progress.update(1)
    progress.update(1)

In [ ]:
print(sampling)
np.abs(ft).max()

In [ ]:
print(sampling)
np.abs(ft).max()

In [ ]:
gvec.comp.ft_autoremove(ft, atol=1e-4)

${g_{\theta_B\theta_B}}_{mn}^c$, `g_tt_booz` -> `g_tt_booz_mnc`

# Visualize output & compare FT

In [ ]:
from ipywidgets import interact, widgets

In [ ]:
def plot_ds(fig, ax, ds, var, dots=True, mode="color"):
    if mode in ["color", "both"]:
        pcm = ax.pcolormesh(ds.zeta_B, ds.theta_B, ds[var], shading="gouraud")
        fig.colorbar(pcm, ax=ax)
    if mode == "contour":
        cnt = ax.contour(ds.zeta_B, ds.theta_B, ds[var], 20)
        fig.colorbar(cnt, ax=ax)
    elif mode == "both":
        ax.contour(ds.zeta_B, ds.theta_B, ds[var], 20, colors="k", linewidths=0.5)

    if dots:
        t, z = np.meshgrid(ds.zeta_B, ds.theta_B)
        ax.plot(t.flat, z.flat, "k.", ms=1)
    ax.set(
        title=f"original ($n_\\theta={ds.theta_B.size}, n_\\zeta={ds.zeta_B.size}$)",
        xlabel=f"${ds.zeta_B.attrs['symbol']}$",
        ylabel=f"${ds.theta_B.attrs['symbol']}$",
    )

In [ ]:
def plot_ft(fig, ax, ft, var, mode="color", ift="fft"):
    nfp = ft.N_FP.item() if "N_FP" in ft else 1
    t, z = (
        np.linspace(0, 2 * np.pi, ft.m.max().item() * 2 + 1, endpoint=False),
        np.linspace(0, 2 * np.pi / nfp, ft.n.max().item() * 2 + 1, endpoint=False),
    )

    varc = f"{var}_mnc"
    vars = f"{var}_mns"
    if varc in ft and vars in ft and ft[varc].dims != () and ft[vars].dims != ():
        c, s = ft[varc].data, ft[vars].data
    elif varc in ft and ft[varc].dims != ():
        c, s = ft[varc].data, np.zeros_like(ft[varc].data)
    elif vars in ft and ft[vars].dims != ():
        c, s = np.zeros_like(ft[vars].data), ft[vars].data
    else:
        print(f"no ft data for {var}")
        return

    if ift == "fft":
        data = gvec.fourier.ifft2d(c, s)
    elif ift == "eval":
        T, Z = np.meshgrid(t, z)
        data = gvec.fourier.eval2d(c, s, T, Z, nfp=nfp).T
    if mode == "color" or mode == "both":
        pcm = ax.pcolormesh(z, t, data, shading="gouraud")
        fig.colorbar(pcm, ax=ax)
    if mode == "contour":
        cnt = ax.contour(z, t, data, 20)
        fig.colorbar(cnt, ax=ax)
    elif mode == "both":
        cnt = ax.contour(z, t, data, 20, colors="k", linewidths=0.5)
    ax.set(
        title=f"Fourier transformed ($M={ft.m.max().item()}, N={ft.n.max().item()}$)",
        # ylabel=f"${ds.theta_B.attrs['symbol']}$",
        xlabel=f"${ds.zeta_B.attrs['symbol']}$",
    )

In [ ]:
def plot(rho, var, dots, mode, ift):
    fig, axs = plt.subplots(
        1, 2, figsize=(10, 5), tight_layout=True, sharex=True, sharey=True
    )
    fig.suptitle(
        f"{var}: ${ds[var].attrs['symbol']}$\n{ds[var].attrs['long_name']}\nBoozer coordinates, one field period, $N_{{FP}}={ds['N_FP'].item()}$, $\\rho={rho:.3f}$"
    )
    plot_ds(fig, axs[0], ds.sel(rho=rho), var, dots, mode)
    plot_ft(fig, axs[1], ft.sel(rho=rho), var, mode, ift)


interact(
    plot,
    rho=widgets.SelectionSlider(options=ds.rho.data),
    var=[var for var in ds.data_vars if ds[var].dims == ("rad", "pol", "tor")],
    dots=[False, True],
    mode=["both", "color", "contour"],
    ift=["fft", "eval"],
);